In [ ]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from models import FBGNet , MultiTaskTransformer, PatchTST, CONFIGS
from datas.FBG_Dataset import min_max_denormalize
from datas import FBGDataset, z_score_normalize_samplewise, min_max_normalize
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import wandb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
x_data = np.loadtxt('./Data_sets/data.txt', delimiter=',')  # (19968, 2000)
# print(x_data.shape)
# 步骤1：重塑数组
x_data = x_data.reshape(-1, 2, 2000)
# 步骤2：调整轴的顺序
normalized_data_x = np.transpose(x_data, (0, 2, 1))
normalized_data_x = z_score_normalize_samplewise(normalized_data_x)  # 忘记归一化了！！

y = np.loadtxt('./Data_sets/label.txt', delimiter=',')  # (9984, 3)

# 假设 x 和 y 是 numpy 数组，需要转换为 PyTorch 的张量
x_tensor = torch.from_numpy(normalized_data_x).float()  # 输入数据
y_direction_tensor = torch.from_numpy(y[:, 0]).long()
y_position_tensor = torch.from_numpy(y[:, 1]).long()
y_force_tensor = torch.from_numpy(y[:,2]).float()

print(x_tensor.shape)
print(y_direction_tensor.shape)
print(y_position_tensor.shape)
print(y_force_tensor.shape)

In [3]:
# 创建数据集实例
fbg_dataset = FBGDataset(x_tensor, y_direction_tensor, y_position_tensor, y_force_tensor, train=False)
test_dataloader = DataLoader(fbg_dataset, batch_size=384, shuffle=False)
import itertools

def get_batch(dataloader, batch_idx):
    return next(itertools.islice(dataloader, batch_idx, batch_idx + 1))


In [ ]:
# model = MultiTaskTransformer(input_dim = 2)
model = PatchTST(num_classes_1=25, num_classes_2=24, configs=CONFIGS)
model.to('cuda')
torch.compile(model) # 编译模型

In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
def test_model(test_dataloader,batch_number, model, model_path):
    batch_N = get_batch(test_dataloader, batch_number)
    inputs, label_direction, labels_position, labels_force = batch_N
    # 加载模型参数
    model.load_state_dict(torch.load(model_path))
    # 将模型设置为评估模式
    model.eval()
    # 进行预测
    with torch.no_grad():
        direction_output,position_output, force_output = model(inputs.to(device))
    # print("Real_Labels(Direction)", label_direction)
    # print("Predicted position:", torch.argmax(direction_output, dim=1))
    # print("Real_Labels(Direction)-Predicted position",label_direction.to('cuda')-torch.argmax(direction_output, dim=1))
    # # print("Real_Labels (Position):", labels_position - 1)
    # print("Predicted position:", torch.argmax(position_output, dim=1))
    # print("Real_Labels (Position)-Predicted position",(labels_position).to('cuda')-torch.argmax(position_output, dim=1))
    print("Real_Labels (Force):", labels_force)
    print("predicted_force:", force_output.cpu().numpy())
    Difference = labels_force.numpy() - force_output.cpu().numpy()
    large_diff_indices = np.where(np.abs(Difference) > 0.5)[0]
    print("索引位置:", large_diff_indices)
    print("对应的真实值:", labels_force.numpy()[large_diff_indices])
    print("对应的预测值:", force_output.cpu().numpy()[large_diff_indices])
    mse_force = mean_squared_error(labels_force.cpu().numpy(), force_output.cpu().numpy())
    mae_force = mean_absolute_error(labels_force.cpu().numpy(), force_output.cpu().numpy())
    print(f'MSE Force: {mse_force:.4f}, MAE Force: {mae_force:.4f}')
    return large_diff_indices


In [ ]:
large_diff_indices = test_model(test_dataloader,0,model,'./optical_fiber_checkpoints/model_both_normalize_newest.pth')


In [ ]:
def data_choose(large_diff_indices,number):    
    x_data[large_diff_indices + 384 * number] = 0
    y[large_diff_indices + 384 * number] = 0
    # print(x_data.shape)
    # print(y.shape)
    # print(x_data[large_diff_indices + 384 * number])
    # print(y[large_diff_indices + 384 * number])


for i in range(26):
    large_diff_indices = test_model(test_dataloader,i,model,'./optical_fiber_checkpoints/model_both_normalize_newest.pth')
    data_choose(large_diff_indices,i)


In [8]:
x_data = x_data.reshape(-1,2000)
np.savetxt('./Data_sets/data_choose_without_noise_0.5.txt', x_data, delimiter=',')
np.savetxt('./Data_sets/label_choose_without_noise_0.5.txt', y, delimiter=',')

In [ ]:
import numpy as np
data_choose = np.loadtxt('./Data_sets/data_choose_without_noise_0.5.txt', delimiter=',')
label_choose = np.loadtxt('./Data_sets/label_choose_without_noise_0.5.txt', delimiter=',')
data_choose = np.delete(data_choose, [0,1,768,769], axis=0)
print(data_choose.shape)
print(label_choose.shape)

In [ ]:
cleaned_arr_data_x = data_choose[~(data_choose == 0).all(axis=1)]
cleaned_arr_label = label_choose[~(label_choose == 0).all(axis=1)]
print(cleaned_arr_data_x.shape)
print(cleaned_arr_label.shape)
np.savetxt('./Data_sets/data_choose_cleaned_without_noise_0.5.txt', cleaned_arr_data_x, delimiter=',')
np.savetxt('./Data_sets/label_choose_cleaned_without_noise_0.5.txt', cleaned_arr_label, delimiter=',')